$$
\renewcommand{\vvec}[2]{\left[ \begin{array}{c} \mathbf{#1}\\ \mathbf{#2} \end{array}\right]}
\renewcommand{\mmat}[4]{\left[ \begin{array}{cc} \mathbf{#1}&\mathbf{#2}\\ \mathbf{#3}&\mathbf{#4} \end{array}\right]}
\renewcommand{\aaa}{\mathbf{a}}
\renewcommand{\AAA}{\mathbf{A}}
\renewcommand{\xyvec}{\left[ \begin{array}{c} \xx\\\yy \end{array} \right]}
\renewcommand{\xyvecc}{\left[ \begin{array}{c} x^1\\y^1 \end{array} \right]}
\renewcommand{\mm}{\mathbf{m}}
\renewcommand{\xx}{\mathbf{x}}
\renewcommand{\yy}{\mathbf{y}}
\renewcommand{\zz}{\mathbf{z}}
\renewcommand{\vv}{\mathbf{v}}
\renewcommand{\ee}{\mathbf{e}}
\renewcommand{\ww}{\mathbf{w}}
\renewcommand{\XX}{\mathbf{X}}
\renewcommand{\YY}{\mathbf{Y}}
\renewcommand{\WW}{\mathbf{W}}
\renewcommand{\VV}{\mathbf{V}}
\renewcommand{\DD}{\mathbf{D}}
\renewcommand{\dd}{\mathbf{d}}
\renewcommand{\ZZ}{\mathbf{Z}}
\renewcommand{\CC}{\mathbf{C}}
\renewcommand{\bbeta}{\boldsymbol{\mathbf{\beta}}}
\renewcommand{\ddelta}{\boldsymbol{\mathbf{\delta}}}
\renewcommand{\mmu}{\boldsymbol{\mathbf{\mu}}}
\renewcommand{\ssigma}{\boldsymbol{\mathbf{\sigma}}}
\renewcommand{\reals}{\mathbb{R}}
\renewcommand{\loglik}{\mathcal{LL}}
\renewcommand{\penloglik}{\mathcal{PLL}}
\renewcommand{\likelihood}{\mathcal{L}}
\renewcommand{\Data}{\textrm{Data}}
\renewcommand{\given}{ \big| }
\renewcommand{\MLE}{\textrm{MLE}}
\renewcommand{\EE}{\mathbb{E}}
\renewcommand{\EEE}{\mathbf{E}}
\renewcommand{\KL}{\textrm{KL}}
\renewcommand{\Bound}{\mathcal{B}}
\renewcommand{\tth}{\textrm{th}}
\renewcommand{\Gaussian}[2]{\mathcal{N}\left(#1,#2\right)}
\renewcommand{\norm}[1]{\left\lVert#1\right\rVert}
\renewcommand{\ones}{\mathbf{1}}
\renewcommand{\corr}[2]{\textrm{corr}(#1,#2)}
\renewcommand{\diag}[1]{\textrm{diag}\left( #1 \right)}
\renewcommand{\sigmoid}[1]{\sigma\left(#1\right)}
\renewcommand{\myexp}[1]{\exp\left\{#1\right\}}
\renewcommand{\mylog}[1]{\log\left\{#1\right\}}
\renewcommand{\argmax}{\mathop{\textrm{argmax}}}
\renewcommand{\new}{\textrm{new}}
\renewcommand{\old}{\textrm{old}}
\renewcommand{\bb}{\mathbf{b}}
\renewcommand{\ba}{\mathbf{a}}
\renewcommand{\bg}{\mathbf{g}}
\renewcommand{\BB}{\mathbf{B}}
\renewcommand{\BA}{\mathbf{A}}
\renewcommand{\BC}{\mathbf{C}}
\renewcommand{\UU}{\mathbf{U}}
\renewcommand{\uu}{\mathbf{u}}
\renewcommand{\hh}{\mathbf{h}}
\renewcommand{\SSS}{\mathbf{S}}
\renewcommand{\sss}{\mathbf{s}}
\renewcommand{\rr}{\mathbf{r}}
\renewcommand{\tr}[1]{\textrm{tr}\left\{#1\right\}}
\renewcommand{\argmin}{\mathop{\textrm{argmin}}}
\renewcommand{\abs}[1]{\left\lvert#1\right\rvert}
\renewcommand{\sign}[1]{\textrm{sign}\left(#1\right)}
\renewcommand{\minimize}{\mathop{\textrm{minimize}}}
\renewcommand{\subjectto}{\mathop{\textrm{subject to}}}
\renewcommand{\relu}{\textrm{ReLU}}
$$

---
# Background
The Federalist Papers is a collection of 85 articles written under pseudonym Publius. The authors of these articles were Alexander Hamilton, James Madison and John Jay. A number of these articles are not clearly attributable to an author. Further, some articles were products of collaboration between authors. 

## Task
We will try to attribute articles to authors. We are going to cluster the articles. To accomplish this, we will use two models.

1. We will model the word counts using mixture of product of Poissons. For each cluster and word combination we will have a single poisson distribution. 
2. We will model transformed word counts using mixture of multivariate Gaussians with diagonal covariance.

We will use articles with known authors to assess our clusters. Specifically, we will perform a statistical test for each cluster and each author that will tell us how likely is it that articles by the same author end up in the cluster by chance. The null hypothesis -- that we are trying to reject -- is that the assignment of articles to clusters is completely random and there is no association between our clusters and authors of articles. We compute probability that the assignment of authors to clusters is generated randomly. If this probability is low, then we can reject the null hypothesis and claim that association between authors and clusters is significant.

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

# load data
try:
    import cPickle as pickle
    kwargs = {}
except:
    import _pickle as pickle
    kwargs = {'encoding':'bytes'}
import gzip
import scipy

with gzip.open("preprocessed_documents.pgz") as f:
    documents, counter = pd.read_pickle(f, compression=None)

dataMat = scipy.sparse.vstack([doc['acounts'] for doc in documents])
dataMat = np.asarray(dataMat.todense().astype('int32'))

We will use counts of functional words as features for clustering the articles. 

In [2]:
# load functional the word list
import codecs
with codecs.open('updated_functionWords.txt') as f:
    function_words = f.read().splitlines() 
# find interaction between the list and the words in the documents
func_lst = np.nonzero(np.in1d(counter.get_feature_names(),(function_words)))[0]
lst = func_lst
words = np.array(counter.get_feature_names())[lst]
print('# of used features: {}, they are {}'.format(len(lst), words))
dataMat_selected = np.asarray(dataMat[:,lst])

# of used features: 199, they are ['about' 'above' 'accordingly' 'after' 'against' 'all' 'along' 'also'
 'although' 'amidst' 'among' 'amongst' 'an' 'and' 'another' 'anti' 'any'
 'anything' 'are' 'around' 'as' 'aside' 'at' 'bar' 'be' 'because' 'been'
 'before' 'behind' 'below' 'beneath' 'besides' 'between' 'beyond' 'both'
 'but' 'by' 'can' 'certain' 'concerning' 'consequently' 'considering'
 'could' 'dare' 'do' 'down' 'during' 'each' 'either' 'enough' 'even'
 'every' 'everything' 'except' 'excluding' 'failing' 'few' 'fewer'
 'following' 'for' 'from' 'given' 'had' 'has' 'have' 'he' 'hence' 'her'
 'hers' 'herself' 'him' 'himself' 'his' 'however' 'if' 'in' 'including'
 'inside' 'into' 'is' 'it' 'its' 'itself' 'less' 'like' 'little' 'many'
 'may' 'me' 'might' 'mine' 'more' 'most' 'much' 'must' 'my' 'myself'
 'near' 'neither' 'nevertheless' 'no' 'none' 'nor' 'not' 'nothing'
 'notwithstanding' 'now' 'of' 'off' 'on' 'once' 'one' 'only' 'opposite'
 'or' 'other' 'ought' 'our' 'ours' 'ourselves' 

---
# Mixture of Poissons 
We will assume that each article belongs to a single cluster.
Since we are modeling word counts, a natural modeling choice is to assume that counts are distributed according to the Poisson distribution. Each cluster will have a Poisson distribution associated with each word.

We will derive the formulation of mixture of Poissons model and fit it to the data.

1) Possion pmf $$p(k \mid \lambda) = \frac{\lambda^{k} e^{-\lambda}}{k!}$$ tells us probability of observing a count of $k$ for specific $\lambda$ value

2) Notation for the model
  1. $x_i$ be the feature vector of the $i^{th}$ sample, $x_{i, j}$ be the $j^{th}$ features of the $i^{th}$ sample. 
  2. $h_i$ be the index of the cluster for the $i^{th}$ sample. 
  3. $\lambda_m$ be the lambda vector for the $m^{th}$ cluster, $\lambda_{m,j}$ be the parameters of Possion pmf for cluster c and feature j.
  4. $p(h_i = c) = \pi_c$
  5. $p(x_i \mid h_i = m, \lambda) = \prod_j p(x_{i,j}\mid \lambda_{m,j})$

3) **[1/2pt]** Write out the log probability of sample $x_i$ given that it belongs to cluster $m$, ($h_i = m$):

$$
\log p(x_i| h_i = m, \lambda) = \sum_j  \log p(x_{i,j}|\lambda_{m,j}) = \sum_j x_{i,j} \log (\lambda_{m,j}) - \lambda_{m,j} - \log (x_{i,j}!)
$$

4) **[1/2pt]** Write out the log of marginal probability of sample $x_i$ in terms of $p(x_i\mid\lambda,h_i)$ and $\pi$

$$
\log p(x_i \mid \lambda,\pi) = \log \sum_c \pi_c p(x_i | h_i = c, \lambda)
$$

5) **[1pt]** Write out the log-likelihood using the above log-probability for all samples.

$$
\begin{aligned}
LL(\lambda,\pi) &= \log\Big( \prod_{i} p(x_{i})\Big)\\
&= \sum_{i}  \log p(x_{i,j}|\lambda_{m,j})\\
& = \sum_{i} \log p(x_i|\lambda,\pi)\\
& = \sum_{i} \log \sum_{c} \pi_{c} p(x_{i}|h_{i} = c,\pi,\lambda)\\
\end{aligned}
$$

6) **[1pt]** Apply Jensen’s inequality to derive a lower-bound.

$$
\begin{aligned}
LL(\lambda, \pi) = \log( \prod_{i} p(x_{i})) &= \sum_{i} \log \left \{ \sum_{c} q(h_i=c) \frac{ p(x_{i}, h_i = c) }{q(h_i=c)} \right \} \\
& \ge \sum_{i} \sum_c q(h_i = c) \log \left\{ \frac{p(x_i, h_i = c)}{q(h_i = c)} \right\} \\
&= \sum_{i} \sum_c q(h_i = c) \log \left\{  p(x_i, h_i = c) \right\} - \sum_{i} \sum_c q(h_i = c)  \log \left\{ q(h_i = c) \right\}
\end{aligned}
$$

To check your answer, plug-in $p(h_i=c \mid x_i)$ in place of $q(h_i=c)$. You should recover the log-likelihood.

7) **[1pt]** If we let $q(h_i)$ be the posterior probability $p(h_i\mid x_i,\lambda, \pi)$, we maximize the lower-bound. Use Bayes rule to derive posterior.

$$
\begin{aligned}
p(h_i = m\mid x_i, \lambda, \pi) &= \frac{p(h_i=m,x_i|\lambda,\pi)}{p(x_i|\lambda,\pi)} \\\\
&= \frac{p(h_i=m|\pi)p(x_i|\lambda,h_i=m)}{\sum_{c}p(h_i=c|\pi)p(x_i|\lambda,h_i=c)}\\\\
&= \frac{\pi_{m}p(x_i|\lambda,h_i=m)}{\sum_{c}\pi_{c}p(x_i|\lambda,h_i=c)}\\\\
\end{aligned}
$$

8) **[1pt]** Implement a function that computes the log probability of a single sample. Note that for computing log factorial you need to use the function we provide.
Due to numerical precision, we compute probability in log domain. First implement the function to compute $\log p(x_i \mid \lambda_m)$

In [3]:
def logsum(lp):
    m = np.max(lp)
    return np.log(np.sum(np.exp(lp-m))) + m

# you cannot compute the factorial directly for large x, compute it in log domain
def logfactorial(x):
    return np.sum(np.log(np.arange(1,x+1)))

# xs: a vector for x_i
# ls: a vector for lambdas
# return log probability of equation 
def logprobvec(xs, ls, compute_factorial = False): 
    logfactorial_val = np.zeros((len(xs,)))
    if compute_factorial:
        for i in np.arange(len(xs)):
            logfactorial_val[i] = logfactorial(xs[i])
    lp = xs*np.log(ls) - ls - logfactorial_val
    lp = np.sum(lp)
    return lp

# test function
test_x = np.array([5, 33, 211, 474])
test_l = np.array([4, 60, 300, 600])
res_v = logprobvec(test_x, test_l, False)
assert(np.allclose(res_v, 3413.687601))

9) **[1pt]** Implement a function that computes the log posterior for all samples.

In [4]:
# xs: an array of shape N*F (N, # samples, F, # features)
# lambdas: an array of shape K*F (K, # clusters, F, # features)
# pis: a vector of shape K
# return three variables:
# logprobs: an array of shape K*N, the log posterior probabilities in (7), the log probability
#           of a sample belong to each cluster
# loglik: log-likelihood of the model in 4)
# labels: a vector of shape N, the most probable cluster each sample belongs to
def logposterior_MP(xs, lambdas, pis):
    K = lambdas.shape[0]
    F = lambdas.shape[1]
    N = xs.shape[0]
    assert(xs.shape[1] == F)
    logprobs = np.zeros((K,N))
    loglik = 0
    labels = np.zeros(N)
    for n in range(N):
        x = xs[n,:]
        for k in range(K):
            ls = lambdas[k,:]
            logprobs[k,n] = np.log(pis[k])+logprobvec(x,ls,False)
        docloglik = logsum(logprobs[:,n])
        loglik = loglik + docloglik
        logprobs[:,n] = logprobs[:,n] - docloglik
        labels[n] = np.argmax(logprobs[:,n])
    return logprobs, loglik, labels

# test function
test_x = np.array([[7, 4, 6], [2, 8, 1], [3, 3, 9]], dtype='float32')
test_l = np.tile(np.round(np.mean(test_x, axis=0)), (3,1))
np.random.seed(10)
test_l = test_l + 1 + np.abs(np.random.randn(test_l.shape[0], test_l.shape[1]))*2
test_pis = np.array([0.33, 0.33, 0.33])
res_lp, res_ll, res_lab = logposterior_MP(test_x, test_l, test_pis)
assert(np.allclose(res_lp, [[-1.57270506, -4.35048081, -2.08895623],
                            [-1.3579241 , -1.1180958 , -0.75511075],
                            [-0.62488554, -0.41521594, -0.90084777]]))
assert(np.allclose(res_ll, 21.969895))
assert(np.allclose(res_lab, [2,2,1]))

10) **[1pt]** Derive the update formulation for $\lambda_{m, j}$. Take the derivative of the lower-bound on log-likelihood.

$$
\begin{aligned}
\frac{\partial}{\partial \lambda_{m,j}} \sum_{i} \sum_{j} \sum_{c} q(h_i=c) \log \left \{  p(x_{i,j}, h_i = c) \right \} &=
\frac{\partial}{\partial \lambda_{m,j}} \sum_{i} \sum_{j} \sum_{c} \log \{ p(h_i=c) \} + \log \{ p(x_{i,j}|h_i=c)\}  \\
&=
\frac{\partial}{\partial \lambda_{m,j}} \sum_{i} \sum_{j} \sum_{c} q(h_i=c) \log\left\{ \frac{\lambda_{m,j}^{x_{i,j}} e^{-\lambda_{m,j}}}{x_{i,j}!}\right\}  \\
&=
\frac{\partial}{\partial \lambda_{m,j}} \sum_{i} \sum_{j} \sum_{c} q(h_i=c) (x_{i,j}\log\{\lambda_{m,j}\}-\lambda_{m,j}-\log\{x_{i,j}!\})  \\
&=
\sum_i\sum_j \Big( \frac{x_{i,j}}{\lambda_{m,j}}-1 \Big)\sum_c q(h_i=c)  \\
&= \sum_{i}  q(h_i=m) \left( \frac{x_{i,j}}{\lambda_{m,j}} - 1 \right)
\end{aligned}
$$

Let the derivative be zero, we have:

$$
\lambda_{m,j} = \frac{ \sum_i q(h_i=m) x_{i,j} }{\sum_i q(h_i=m)}
$$

11) **[1pt]** Derive the update formulation for $\pi_m$. Take the derivative of the lower-bound. $\big($Note: $\sum_c \pi_c = 1\big)$

$$
\text{Lagrangian}\> L(\pi, \gamma) = \sum_{i}  q(h_i=m) \log \left\{ \pi_m \right\} + \gamma\Big(\sum_c \pi_c - 1\Big)
$$

Taking derivatives on both $\pi_m$ and $\gamma$ and setting them to zero, we have:

$$
\begin{aligned}
    &\sum_i \frac{q(h_i=m)}{\pi_m} + \gamma = 0 \\
    &\sum_c \pi_c= 1
\end{aligned}
$$

From above, we have:

$$
\pi_m = \frac{ \sum_i q(h_i=m) }{N}
$$

where N is the number of samples.

12) **[1pt]**  Write the code to update $\lambda_{m,j}$ and $\pi_m$ 

In [5]:
# qs: an array of shape K*N, the posterior probabilities in (7), the probability of a sample belong to each cluster
# return two variables:
# ls: the updated lambda, an array of shape K*F (K, # clusters, F, # features)
# pis: the updated pi, an vector of shape K
def update_MP(qs, xs):
    suff = np.dot(qs,xs)
    # we add small constant to avoid division by zero
    ls = (suff+0.001)/(0.001+np.sum(qs,1)[:,np.newaxis])
    pis = np.sum(qs,axis=1)
    pis = pis/np.sum(qs)
    return ls, pis

# test function
test_qs = np.array([[.9, .3],
                    [.05, .4],
                    [.05, .3]])
test_xs = np.array([[7,4,6], [2,8,1]], dtype='float32')
res_ls, res_pis = update_MP(test_qs, test_xs)
assert(np.allclose(res_ls,[[5.746044, 4.996669, 4.74687 ],
                            [2.552106, 7.541019, 1.554323],
                            [2.709401, 7.410256, 1.712250]]))
assert(np.allclose(res_pis, [0.6, 0.225, 0.175]))

13)  Run the code below, which will invoke your E-step (```logposterior_MP```) and M-step (```update_MP```)

In [6]:
from collections import Counter
def report_labels(labels, side_info):
    for l in np.unique(labels):
        print('cluster {0:d}'.format(int(l)))
        members = np.nonzero(labels==l)
        tmp = np.sort(np.array(side_info)[members])
        tmp = Counter(tmp)
        for z in tmp.keys():
            author = z
            if z == '':
                author = 'DISPUTED'
            print('({} article(s) by {}),'.format(tmp[z], author), end=' ')
        print()

def fit(xs, K, side_info):
    L = xs.shape[1]
    N = xs.shape[0]
    print("Fitting on data of size N: {} and L: {}".format(N, L))
    best_loglik = -1e+308
    best_logliks = []
    best_labels = []
    best_ls = []
    best_pis = []
    verbose = False
    for s in range(100): # run the algorithm 100 times
        np.random.seed(s)
        ls = np.mean(xs,0)*(1.0 + 0.5*(np.random.rand(K,L)-0.5)) # each time initialize with different lambda
        pis = [1./K]*K
        logliks = []
        for it in range(50): # each time run for 50 iterations
            logqs, loglik, labels = logposterior_MP(xs, ls, pis)
            qs = np.exp(logqs)
            logliks.append(loglik)
            ls,pis = update_MP(qs,xs)
        if verbose:
            print('\n Seed {}: log-likelihood = {:.5}, best log-likelihood so far = {:.5}'.format(s+1, loglik, best_loglik))
        if loglik > best_loglik:
            best_loglik = loglik
            print('\n A fit with better log-likelihood ({}) found for seed {}'.format(loglik, s))
            report_labels(labels,side_info)
            best_ls = ls
            best_pis = pis
            best_labels = labels
            best_logliks = logliks
    print('Best')
    report_labels(best_labels,side_info)
    return best_ls, best_pis, best_labels, best_logliks

ls_MP, pis_MP, labels_MP, logliks_MP = fit(dataMat_selected, 4, [doc['authors'] for doc in documents])  

Fitting on data of size N: 85 and L: 199

 A fit with better log-likelihood (250570.84708075694) found for seed 0
cluster 0
(2 article(s) by DISPUTED), (18 article(s) by HAMILTON), 
cluster 1
(8 article(s) by DISPUTED), (9 article(s) by HAMILTON), (1 article(s) by JAY), (3 article(s) by MADISON), 
cluster 2
(1 article(s) by DISPUTED), (10 article(s) by HAMILTON), (11 article(s) by MADISON), 
cluster 3
(4 article(s) by DISPUTED), (14 article(s) by HAMILTON), (4 article(s) by JAY), 

 A fit with better log-likelihood (250708.9882702828) found for seed 1
cluster 0
(4 article(s) by DISPUTED), (11 article(s) by HAMILTON), (4 article(s) by JAY), 
cluster 1
(1 article(s) by DISPUTED), (8 article(s) by HAMILTON), (4 article(s) by MADISON), 
cluster 2
(3 article(s) by DISPUTED), (26 article(s) by HAMILTON), (1 article(s) by MADISON), 
cluster 3
(7 article(s) by DISPUTED), (6 article(s) by HAMILTON), (1 article(s) by JAY), (9 article(s) by MADISON), 
Best
cluster 0
(4 article(s) by DISPUTED), (1

---
# Another Model

In this model we first re-weight our features with a method called **tf-idf**. The main idea of **tf-idf** is that a word which appears in most of the documents, provides less information for classification/clustering; and, therefore, this word should be weighted down.

1. **tf(n, f)**: term frequency(counts), in our case, it is the value of the feature(word) f in the document n.


2. **df(f)**: document frequency, the number of documents which contain word f.


3. **idf(f)**: inverse document-frequency,

$$
\text{idf}(f)=\log \frac{1+N}{1+df(f)} + 1
$$

where N is the number of samples (documents).


4. **tf-idf** is defined as 

$$
\text{tf-idf}(n, f) = \text{tf}(n, f)*\text{idf}(f)
$$

5. Each sample after **tf-idf** transformation is normalized with their Euclidean norm

$$
x_i = \frac{x_i}{ \Vert {x_i} \Vert_2 }
$$

1) **[1pt]** Implement tf-idf computation.

In [7]:
def compute_idf(dataMat):
    idfVec = np.zeros((dataMat.shape[1],))
    N = dataMat.shape[0]
    for i in np.arange(dataMat.shape[1]):
        df = np.count_nonzero(dataMat[:,i]>0, axis=0)
        idfVec[i] = np.log((1+N)/(1+df))+1
    return idfVec
test_data = np.array([[1,0,1,0], [0,1,0,0], [1,1,3,0], [2,3,0,4]])
res_idf = compute_idf(test_data)
assert(np.allclose(res_idf, [1.223143, 1.223143, 1.510825, 1.916290]))

def compute_tfidf(dataMat):
    data_idf = compute_idf(dataMat)
    N = dataMat.shape[0]
    dataMat_tfidf = np.zeros(dataMat.shape, dtype='float32')
    for i in np.arange(N):
        dataMat_tfidf[i,:] = dataMat[i,:]*data_idf
        dataMat_tfidf[i,:] = dataMat_tfidf[i,:]/np.sqrt(np.sum(dataMat_tfidf[i,:]**2))
    return data_idf, dataMat_tfidf

# test function
test_data = np.array([[1,0,1,0], [0,1,0,0], [1,1,3,0], [2,3,0,4]], dtype='float32')
data_idf, res_tfidf = compute_tfidf(test_data)
assert(np.allclose(res_tfidf, [[0.62922752, 0.        , 0.77722114, 0.],
                               [0.        , 1.        , 0.        , 0.],
                               [0.25212485, 0.25212485, 0.93427306, 0.],
                               [0.27662638, 0.41493958, 0.        , 0.86677736]]))

We will now transform the data to tf-idf


In [8]:
data_idf, dataMat_tfidf = compute_tfidf(dataMat)
dataMat_tfidf_selected = np.asarray(dataMat_tfidf[:,lst])

---
# Mixture of Gaussian Model

Since the data after tf-idf transformed is no longer count data, we should model to Gaussian.

1) Multivariate Gaussian pdf

$$
p(x\mid\mu, \Sigma) = (2 \pi)^{-\frac{k}{2}} |\Sigma|^{-\frac{1}{2}} \exp \{ -\frac{1}{2} {(x-\mu)^T \Sigma^{-1} (x-\mu)} \}
$$

In our model, we set $\Sigma$ to be a diagonal matrix.

2) Notation for the model:
  1. $x_i$ be the feature vector of the $i^{th}$ sample, $x_{i, j}$ be the $j^{th}$ features of the $i^{th}$ sample. 
  2. $h_i$ be the index of the cluster for the $i^{th}$ sample. 
  3. $\mu_m$ be the mean vector for the $m^{th}$ cluster, $\mu_{m,j}$ be the $j^{th}$ feature of the mean vector.
  4. $\Sigma_m$ be the covariance matrix for the $m^{th}$ cluster, $\sigma^2_{m, j}$ be the $j^{th}$ variance for the $m^{th}$ cluster.
  5. $p(h_i = c) = \pi_c$
  
3)
$$
\begin{aligned}
\log p(x_i\mid \mu_m, \Sigma_m ) &= -\frac{k}{2}\log(2\pi)-\frac{1}{2}\log(|\Sigma_m|)-\frac{1}{2}(x_i-\mu_m)^T\Sigma_m^{-1}(x_i-\mu_m) \\
&= -\frac{k}{2}\log(2\pi)-\frac{1}{2}\log\Big(\prod_j \sigma^2_{m,j}\Big)-\frac{1}{2} \sum_j \frac{(x_{i,j}-\mu_{m,j})^2}{\sigma_{m,j}^2}
\end{aligned}
$$

Questions 4-6 are equivalent to the answers you gave in mixture of Poissons. Of course, this model uses different parameters. Hence instead of $\lambda_m$ you would write $\mu_m$ and $\sigma_m^2$

4) Log-likelihood of mixture of Gaussian model is:

$$
\begin{aligned}
LL(\mu,\sigma) &= \sum_i \log \sum_c \pi_c \left[-\frac{k}{2}\log(2\pi)-\frac{1}{2}\log(\sigma^2_{i})-\frac{1}{2} \frac{(x_{i}-\mu_{i})^2}{\sigma_{i}^2}\right] \\
\end{aligned}
$$

5) Apply Jensen’s inequality to derive lower-bound.

$$
\begin{aligned}
LL(\mu, \sigma^2, \pi) = \log\Big( \prod_{i} p(x_{i})\Big) &= \sum_{i} \sum_{h_i} q(h_i = m)\left[ -\frac{k}{2} \log (2\pi)-{\frac{1}{2}\log(\sigma^2_{i})}\right.\left. - \frac{(x_{i}-\mu_{i})^2}{2\sigma_{i}^2} \right]\\
&- \sum_{i}\sum_{h_i}q(h_i = m)\log q(h_i = m)
\end{aligned}
$$

6) If we let $q(h_i)$ be the posterior probability $p(h_i \mid x_i, \mu,\sigma,\pi)$, we maximize the lower-bound. Derive posterior formulation. 

$$
\begin{aligned}
p(h_i = m\mid x_i, \mu,\sigma^2,\pi) &= \frac{\pi_m\big(-\frac{k}{2}\log(2\pi)-\frac{1}{2}\log(|\Sigma_m|)-\frac{1}{2}(x_i-\mu_m)^T\Sigma_m^{-1}(x_i-\mu_m)\big)}{\sum_c\pi_c\big(-\frac{k}{2}\log(2\pi)-\frac{1}{2}\log(|\Sigma_m|)-\frac{1}{2}(x_i-\mu_m)^T\Sigma_m^{-1}(x_i-\mu_m)\big)}
\end{aligned}
$$

7) **[1pt]**  Write the code to compute posterior. Due to numerical precision, we compute probability in log domain. First implement the function to compute $\log p(x_i \mid \mu_m,\sigma_m^2)$

In [9]:
# xs: a vector for x_i
# mu: a vector for mu
# sigma2: a vector for sigma square. Since we assume covariance matrix is diagonal,
#         we can use a vector to save the non-zero values
# return log probability
def logprobvec_MG(xs, mu, sigma2):
    lp = (-0.5*mu.shape[0])*np.log(2.0*np.pi)-0.5*np.sum(np.log(sigma2))-0.5*np.sum((xs-mu)**2.0/(sigma2))
    return lp

# test function
test_x = np.array([.2, 2.2])
test_mu = np.array([1, 3])
test_sigma2 = np.array([2, 0.5])
res_v = logprobvec_MG(test_x, test_mu, test_sigma2)
assert(np.allclose(res_v, -2.6378770))

In [10]:
# xs: an array of shape N*F (N, # samples, F, # features)
# mus: an array of shape K*F (K, # clusters, F, # features)
# sigma2s: an array of shape K*F (K, # clusters, F, # features)
# pis: a vector of shape K
# return three variables:
# logprobs: an array of shape K*N, the log posterior probabilities in (7), the log probability
#           of a sample belong to each cluster
# loglik: log-likelihood of the model in (4)
# labels: a vector of shape N, the most probable cluster each sample belongs to
def logposterior_MG(xs, mus, sigma2s, pis):
    K = mus.shape[0]
    F = mus.shape[1]
    N = xs.shape[0]
    assert(xs.shape[1] == F)
    logprobs = np.zeros((K,N))
    loglik = 0
    labels = np.zeros(N)
    for n in range(N):
        x = xs[n,:]
        for k in range(K):
            mu = mus[k,:]
            sigma2 = sigma2s[k,:]
            logprobs[k,n] = np.log(pis[k]) + logprobvec_MG(x,mu,sigma2)
        docloglik = logsum(logprobs[:,n]+1e-6)
        loglik = loglik + docloglik
        logprobs[:,n] = logprobs[:,n] - docloglik
        labels[n] = np.argmax(logprobs[:,n])
    return logprobs, loglik, labels

# test function
test_x = np.array([[1.2,.2,.1], [1,2,1], [.5,.6,1]], dtype='float32')
test_mu = np.tile(np.round(np.mean(test_x, axis=0)), (3,1))
np.random.seed(10)
test_mu = test_mu + np.random.randn(test_mu.shape[0], test_mu.shape[1])
test_sigma = np.ones(test_mu.shape)
test_pis = np.array([0.33, 0.33, 0.33])
res_lp, res_ll, res_lab = logposterior_MG(test_x, test_mu, test_sigma, test_pis)
assert(np.allclose(res_lp, [[-1.91883423, -2.51793008, -3.58124658],
                            [-0.97027882, -0.72769468, -0.98950581],
                            [-0.74603191, -0.82930499, -0.51016141]]))
assert(np.allclose(res_ll, -11.189608))
assert(np.allclose(res_lab, [2,1,2]))

8) **[1pt]** Derive the update formulation for $\mu_{m,j}$. Take the derivative of the lower-bound.

$$
\begin{aligned}
\frac{\partial}{\partial \mu_{m, j}} \sum_{i} \sum_{c} q(h_i=c) \log \left \{  p(x_{i}, h_i = c) \right \} = \sum_{i} q(h_i = c)(x_{i,j}-\mu_{m,j})
\end{aligned}
$$

Let the derivative be zero, we have:

$$
\mu_{m, j} = \frac{\sum_i q(h_i = c) x_{i,j}}{\sum_i q(h_i = c)}
$$

9) **[1pt]** Derive the update formulation for $\sigma^2_{m,j}$. Take the derivative on the lower-bound derived from (3).

$$
\begin{aligned}
\frac{\partial}{\partial \sigma^2_{m, j}} \sum_{i} \sum_{c} q(h_i=c) \log \left \{  p(x_{i}, h_i = c) \right \} &=
\sum_i q(h_i = c) \left( - \frac{1}{2 \sigma^2_{m,j}} + \frac{(x_{i,j} - \mu_{m,j})^2}{ 2(\sigma^2_{m,j})^2 }\right)
\end{aligned}
$$

Let the derivative be zero, we have:

$$
\sigma^2_{m, j} = \frac{\sum_i q(h_i = c) (x_{i,j}-\mu_{m,j})^{2}}{\sum_i q(h_i = c)}
$$

10) Update formulation for $\pi_m$ is the same as mixture of Poisson model

$$
\pi_m = \frac{\sum_i q(h_i = m)}{N}
$$

where N is the number of samples.

11) **[1pt]** Write the code to update $\mu_{m,j}$, $\sigma^2_{m,j}$ and $\pi_m$ (hint: the update rule of $\pi_m$ is the same as mixture of Poisson).

In [11]:
# qs: an array of shape K*N, the posterior probabilities in (7), the probability of a sample belong to each cluster
# xs: an array of shape N*F (N, # samples, F, # features)
# return three variables:
# mus: the updated \mu, an array of shape K*F (K, # clusters, F, # features)
# sigma2s: the updated \sigma^2, an array of shape K*F (K, # clusters, F, # features)
# pis: the updated \pi, a vector of shape K
def update_MG(qs, xs):
    suff = np.dot(qs,xs)
    denom = np.sum(qs,1)[:,np.newaxis]
    mus = (suff+1e-6)/(denom+1e-6)
    K = qs.shape[0]
    sigma2s = np.zeros(mus.shape)
    for i in np.arange(K):
        suff = np.dot(qs[i,:],(xs - mus[i,:])**2)
        sigma2s[i] = (suff+1e-6)/(np.sum(qs[i,:])+1e-6)
    sigma2s = sigma2s+1e-6
    pis = np.sum(qs,axis=1)
    pis = pis/np.sum(qs)
    return mus, sigma2s, pis

# test function
test_qs = np.array([[0.14677797, 0.08062632, 0.02784097],
                    [0.37897736, 0.48302123, 0.37176037],
                    [0.47424467, 0.43635245, 0.60039866]])
test_xs = np.array([[1.2,.2,.1], [1,2,1], [.5,.6,1]], dtype='float32')
res_mus, res_sigma2s, res_pis = update_MG(test_qs, test_xs)
assert(np.allclose(res_mus, [[1.060471, 0.812210, 0.482459],
                             [0.910773, 1.025236, 0.723544],
                             [0.864096, 0.878753, 0.717524]]))
assert(np.allclose(res_sigma2s, [[0.04661863, 0.66609716, 0.1979422 ],
                                 [0.07965803, 0.63567057, 0.17238403],
                                 [0.09342444, 0.53853368, 0.17443729]]))
assert(np.allclose(res_pis, [0.085081, 0.411252, 0.503665]))

In [12]:
from collections import Counter
def fit_MG(xs,K,side_info):
    F = xs.shape[1]
    N = xs.shape[0]
    print("Fitting on data of size N: {} and F: {}".format(N, F))
    best_loglik = -1e+308
    best_logliks = []
    best_labels = []
    best_mu = []
    best_sigma2s = []
    best_pis = []
    verbose = False
    print()
    for s in range(100): # run the algorithm 100 times
        np.random.seed(s)
        mus = np.mean(xs,0) + .01*np.random.randn(K,F) # each time initialize with different lambda
        sigma2s = np.ones((K,F))*10
        pis = [1./K]*K
        logliks = []
        for it in range(20): # each time run for 50 iterations
            logqs,loglik, labels = logposterior_MG(xs, mus, sigma2s, pis)
            qs = np.exp(logqs)
            logliks.append(loglik)
            mus, sigma2s, pis = update_MG(qs,xs)
        if verbose:
            print('\n Seed {}: log-likelihood = {:.5}, best log-likelihood so far = {:.5}'.format(s+1, loglik, best_loglik))
        print("run "+str((s+1)))
        if loglik > best_loglik:
            best_loglik = loglik
            print('\n A fit with better log-likelihood ({}) found for seed {}'.format(loglik, s))
            report_labels(labels,side_info)
            best_mus = mus
            best_sigma2s = sigma2s
            best_pis = pis
            best_labels = labels
            best_logliks = logliks
    print('Best')
    report_labels(best_labels,side_info)
    return best_mus, best_sigma2s, best_pis, best_labels, best_logliks

mus_MG, sigma2s_MG, pis_MG, labels_MG, logliks_MG = fit_MG(dataMat_tfidf_selected, 4, [doc['authors'] for doc in documents])

Fitting on data of size N: 85 and F: 199

run 1

 A fit with better log-likelihood (63648.389060799775) found for seed 0
cluster 0
(1 article(s) by DISPUTED), (5 article(s) by HAMILTON), (2 article(s) by JAY), (1 article(s) by MADISON), 
cluster 1
(9 article(s) by DISPUTED), (5 article(s) by HAMILTON), (1 article(s) by JAY), (8 article(s) by MADISON), 
cluster 2
(2 article(s) by DISPUTED), (4 article(s) by HAMILTON), (2 article(s) by JAY), 
cluster 3
(3 article(s) by DISPUTED), (37 article(s) by HAMILTON), (5 article(s) by MADISON), 
run 2

 A fit with better log-likelihood (63680.56234214898) found for seed 1
cluster 0
(1 article(s) by DISPUTED), (30 article(s) by HAMILTON), (1 article(s) by MADISON), 
cluster 1
(5 article(s) by DISPUTED), (3 article(s) by HAMILTON), (3 article(s) by JAY), (4 article(s) by MADISON), 
cluster 2
(9 article(s) by DISPUTED), (12 article(s) by HAMILTON), (9 article(s) by MADISON), 
cluster 3
(6 article(s) by HAMILTON), (2 article(s) by JAY), 
run 3

 A fit

---
# Compare Two Clustering Results With Hypergeometric Test

Next we will test each cluster for enrichment in articles authored by different people (Hamilton, Madison, Jay). If the p-value is small, then the level of enrichment could not have occurred simply by chance.

In [13]:
from scipy.stats import hypergeom
def gen_table(labels, documents):
    K = len(np.unique(labels))
    authorList = np.asarray([doc['authors'] for doc in documents])
    nameList = np.asarray(['JAY', 'MADISON', 'HAMILTON'])
    nameList2 = np.asarray(['JAY', 'MADISON', 'HAMILTON', ''])
    nameNum = {'JAY': 5, 'MADISON': 14, 'HAMILTON': 51}
    enrichment = np.zeros((K,3))
    numberTab = np.zeros((K,4))
    for i in np.arange(K):
        cList = authorList[np.nonzero(labels==i)[0]]
        cnt = 0
        for j in nameList:
            rv = hypergeom(70, nameNum[j], len(cList))
            cIntersect = len(np.nonzero(cList==j)[0])
            enrichment[i,cnt] = 1-rv.cdf(cIntersect)
            numberTab[i,cnt] = cIntersect
            cnt = cnt + 1
        cnt = 0
        for j in nameList2:
            cIntersect = len(np.nonzero(cList==j)[0])
            numberTab[i, cnt] = cIntersect
            cnt = cnt + 1
    return enrichment, numberTab

def report_table(enrichment, numberTab):
    nameList = np.asarray(['JAY', 'MADISON', 'HAMILTON'])
    nameList2 = np.asarray(['JAY', 'MADISON', 'HAMILTON', 'DISPUTED'])
    print('p-value (smaller indicates more significant enrichment): ')
    print('\t\t', end="")
    for z in nameList:
        print('{0:<8}\t'.format(z), end='')
    print()
    for i in range(4):
        print('cluster {}\t'.format(i), end='')
        for j in range(3):
            print('{0:3f}\t'.format(enrichment[i,j]), end='')
        print()

    print('\n\ncount table: ')
    print('\t\t', end="")
    for z in nameList2:
        print('{0:<8}\t'.format(z), end='')
    print()
    for i in range(4):
        print('cluster {}\t'.format(i), end='')
        for j in range(4):
            print('{0:3f}\t'.format(numberTab[i,j]), end='')
        print()

In [14]:
print('Clustering Results of Mixture of Poissons:')
enrichment, numberTab = gen_table(labels_MP, documents)
report_table(enrichment, numberTab)

Clustering Results of Mixture of Poissons:
p-value (smaller indicates more significant enrichment): 
		JAY     	MADISON 	HAMILTON	
cluster 0	0.000961	0.993311	0.919420	
cluster 1	0.654044	0.077191	0.754442	
cluster 2	0.945633	0.998012	0.004683	
cluster 3	0.534303	0.001177	1.000000	


count table: 
		JAY     	MADISON 	HAMILTON	DISPUTED	
cluster 0	4.000000	0.000000	11.000000	4.000000	
cluster 1	0.000000	4.000000	8.000000	1.000000	
cluster 2	0.000000	1.000000	26.000000	3.000000	
cluster 3	1.000000	9.000000	6.000000	7.000000	


In [25]:
print('Clustering Results of Mixture of Gaussian:')
enrichment, numberTab = gen_table(labels_MG, documents)
report_table(enrichment, numberTab)

Clustering Results of Mixture of Gaussian:
p-value (smaller indicates more significant enrichment): 
		JAY     	MADISON 	HAMILTON	
cluster 0	0.929714	0.000008	1.000000	
cluster 1	0.000000	0.700810	0.993397	
cluster 2	0.983361	0.999998	0.000000	
cluster 3	0.465340	0.505597	0.067451	


count table: 
		JAY     	MADISON 	HAMILTON	DISPUTED	
cluster 0	0.000000	12.000000	3.000000	13.000000	
cluster 1	5.000000	1.000000	4.000000	1.000000	
cluster 2	0.000000	0.000000	37.000000	1.000000	
cluster 3	0.000000	1.000000	7.000000	0.000000	


12) **[1pt]** State your conclusion about who is most likely to have authored disputed articles.

Based on the p-values from both models, it seems **Madison** is most likely to have authored disputed articles. That is to <br> say, Madison has the smallest p-value $(0.001177)$ for the mixture of Poissons cluster $($number $3)$ where most of the <br> disputed values are found, and also the smallest p-value $(0.000008)$ for the mixture of Gaussian cluster $($number $0)$, <br> where most of the disputed values are found.